In [2]:
import pandas as pd
import numpy as np
import re
import csv
import spacy

nlp = spacy.load('en_core_web_lg')
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


Exploring and cleaning the Data set

In [3]:
full_df=pd.read_csv("complaint_data.csv",on_bad_lines='skip',quoting=csv.QUOTE_NONE)
full_df.head()


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,08/09/2015,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,Company chooses not to provide a public response,Experian Information Solutions Inc.,NJ,08872,NaN,Consent not provided,Web,08/09/2015,Closed with non-monetary relief,Yes,No,1509954
1,12/23/2019,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,NaN,NaN,AES/PHEAA,MA,019XX,NaN,NaN,Web,12/23/2019,Closed with explanation,Yes,NaN,3475943
2,03/04/2016,Credit card,NaN,Billing disputes,NaN,I am dissatisfied with the current outcome of ...,NaN,DISCOVER BANK,NV,891XX,NaN,Consent provided,Web,03/04/2016,Closed with explanation,Yes,Yes,1816726
3,12/28/2016,Credit reporting,NaN,Incorrect information on credit report,Account terms,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,GA,31069,NaN,Consent not provided,Web,12/28/2016,Closed with explanation,Yes,Yes,2266132
4,01/08/2019,"""Credit reporting",credit repair services,"or other personal consumer reports""",Credit reporting,Incorrect information on your report,Public record information inaccurate,"""I have a public record & Child Support entry ...",supervisors,and managers in regards to the erroneous publ...,I called and spoke with a representative and ...,XXXX and Experian verifies their public recor...,the information is sent back to the credit bu...,I will need to contact them with my name and ...,I know the court system and their procedures,but decided to call anyway.,NaN,NaN


In [4]:
df_cleaned=full_df
df_cleaned.columns


Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [5]:
#cleaning data
df_cleaned = df_cleaned.dropna(subset=['Consumer complaint narrative'])
df_cleaned = df_cleaned.drop((df_cleaned.loc[df_cleaned['Consumer complaint narrative'].str.len()<10]).index)



In [6]:
#checking date 
date_pattern = r'\d{2}/\d{2}/\d{4}'
valid_dates=df_cleaned['Date received'].str.contains(date_pattern, regex=True,na=False)
df_cleaned = df_cleaned[valid_dates]
df_cleaned.head()
df_cleaned = df_cleaned.reset_index(drop=True)

I think this data is extracted by scraping. So to get all the issue in one column i will have to concat some columns in Consumer Complaint Narrative.

In [7]:
df_cleaned=df_cleaned[['Product','Consumer complaint narrative', 'Company public response', 'Company response to consumer']]
df_cleaned['final_problem']=df_cleaned['Consumer complaint narrative']

wrong_public_response=df_cleaned.loc[df_cleaned['Company public response'].str.len()>100]
df_cleaned.loc[wrong_public_response.index, 'final_problem'] =wrong_public_response['final_problem']+wrong_public_response['Company public response']


In [8]:
df_cleaned.head()

,Product,Consumer complaint narrative,Company public response,Company response to consumer,final_problem
0,Credit card,I am dissatisfied with the current outcome of ...,NaN,Closed with explanation,I am dissatisfied with the current outcome of ...
1,"""Credit reporting",Incorrect information on your report,Public record information inaccurate,I know the court system and their procedures,Incorrect information on your report
2,Bank account or service,"""On XXXX/XXXX/2015 my bank account balance wit...",NaN,NaN,"""On XXXX/XXXX/2015 my bank account balance wit..."
3,Debt collection,"""I received a collection notice and called rig...",much older than the notice I called about. I ...,NaN,"""I received a collection notice and called rig..."
4,Credit card or prepaid card,"""I had 3 accounts with Chase. They closed my a...",NaN,NaN,"""I had 3 accounts with Chase. They closed my a..."


In [9]:
df_cleaned=df_cleaned.drop(columns=['Consumer complaint narrative','Company public response', 'Company response to consumer'])

After getting the rows which we are gonna work upon, we are gonna prepare the text for topic modelling.
Make the text lowercase
Remove text in square brackets
Remove punctuation
Remove words containing numbers


In [10]:
#preprocessing data
def clean_data(text):
    text = text.lower() # text to lowercase
    text = re.sub(r'\s\{\$\S*', '',text) # Remove text within curly braces
    text = re.sub(r'\n', '', text) # Remove line breaks
    text = re.sub(r'\(\w*\)', '', text) #remove text within braces
    text = re.sub(r'(\W\s)|(\W$)|(\W\d*)', ' ',text) # Remove punctuation
    text = re.sub(r'x+((/xx)*/\d*\s*)|x*', '',text) #Remove date
    text = re.sub(r'\d+\s', '', text) #Remove other numerical values
    text = re.sub(r' +', ' ',text) #Remove unnecessary white spaces
    return text

In [11]:
pd.set_option('display.max_colwidth', None)
df_cleaned['final_problem'] = df_cleaned['final_problem'].apply(clean_data)
df_cleaned.head(20)

,Product,final_problem
0,Credit card,i am dissatisfied with the current outcome of a dispute that was initiated with discover card regarding a single transaction that occurred on in the amount of i have corresponded with discover card at least four times since which i have enclosed as an attachment to this complaint i believe that the credit card issuer has violated consumer protection laws by failing to implement the special rule for credit card purchase protection despite overwhelming paperwork evidence submitted by me that shows the merchant has conducted business in bad faith less favorable to the consumer i have sustained a monetary loss as a result of merchants bad faith and intent i have patiently utilized the internal discover card dispute process over the past three months with the credit card issuer always favoring the merchant i have repeatedly submitted irrefutable paperwork evidence that has shown that the merchant has conducted business in bad faith i have tried in good faith to address my complaint with the merchant and discover card but believe that i will not receive a favorable outcome
1,"""Credit reporting",incorrect information on your report
2,Bank account or service,on my bank account balance with pnc bank was on the bank charged a overdraft fee with no other posted transactions
3,Debt collection,i received a collection notice and called right away i figured i had over looked it as we have many medical bills while on the phone i was told there were additional much older than the notice i called about i paid the one notice and said i needed to look at the date for the other one one of those were for my adult son and could not be discussed with me a separate complaint is being filed
4,Credit card or prepaid card,i had accounts with chase they closed my accounts upon review for reasons
5,Mortgage,on or about through
6,Debt collection,national recovery
7,Credit reporting,because i filed bankruptcy and included the derogatory items are already in with the bankruptcy they are shown twice in different areas and should be removed
8,Credit card,i have apply for credit card at brclysbank de but i was turn down in the report i send you with the attachment on page at the top brclybank de is on my report as a creditor i did not get any credit card i have call the company and they say something about deletion letter
9,Credit reporting,hello i try and keep track of the items that are on my credit especially my inquiries but yesterday i noticed that i have some inquiries that do not belong to me i have n t lost any of my information i do n t think and i do n t believe someone has stolen my identity please send me information regarding these inquiries or have them removed as soon as possible because it is truly hurting my credit and i do n t have these accounts


In [12]:
df_cleaned.to_csv('db_cleaned.csv', index=False)

In [13]:
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))
# print(stop_words)

def remove_stopwords(text):
  
    word_tokens = word_tokenize(text)
    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    sentence = ' '.join(filtered_sentence)
    doc = nlp(sentence)
 
    # Extract lemmatized tokens
    lemmatized_tokens = [token.lemma_ for token in doc]

    # Join the lemmatized tokens into a sentence
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text


df_cleaned['lemmatised_description']=""

for i, row in df_cleaned.iterrows():
    lemmatized_text = remove_stopwords(row['final_problem'])
    df_cleaned.loc[i, 'lemmatised_description'] = lemmatized_text
    
# print(remove_stopwords(df_cleaned['final_problem']))
# df_cleaned['lemmatised_description']=remove_stopwords(df_cleaned['final_problem'])
df_cleaned.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Product,final_problem,lemmatised_description
0,Credit card,i am dissatisfied with the current outcome of a dispute that was initiated with discover card regarding a single transaction that occurred on in the amount of i have corresponded with discover card at least four times since which i have enclosed as an attachment to this complaint i believe that the credit card issuer has violated consumer protection laws by failing to implement the special rule for credit card purchase protection despite overwhelming paperwork evidence submitted by me that shows the merchant has conducted business in bad faith less favorable to the consumer i have sustained a monetary loss as a result of merchants bad faith and intent i have patiently utilized the internal discover card dispute process over the past three months with the credit card issuer always favoring the merchant i have repeatedly submitted irrefutable paperwork evidence that has shown that the merchant has conducted business in bad faith i have tried in good faith to address my complaint with the merchant and discover card but believe that i will not receive a favorable outcome,dissatisfied current outcome dispute initiate discover card regard single transaction occur amount correspond discover card least four time since enclose attachment complaint believe credit card issuer violate consumer protection law fail implement special rule credit card purchase protection despite overwhelming paperwork evidence submit show merchant conduct business bad faith less favorable consumer sustain monetary loss result merchant bad faith intent patiently utilize internal discover card dispute process past three month credit card issuer always favor merchant repeatedly submit irrefutable paperwork evidence show merchant conduct business bad faith try good faith address complaint merchant discover card believe receive favorable outcome
1,"""Credit reporting",incorrect information on your report,incorrect information report
2,Bank account or service,on my bank account balance with pnc bank was on the bank charged a overdraft fee with no other posted transactions,bank account balance pnc bank bank charge overdraft fee post transaction
3,Debt collection,i received a collection notice and called right away i figured i had over looked it as we have many medical bills while on the phone i was told there were additional much older than the notice i called about i paid the one notice and said i needed to look at the date for the other one one of those were for my adult son and could not be discussed with me a separate complaint is being filed,receive collection notice call right away figure look many medical bill phone tell additional much old notice call pay one notice say need look date one one adult son could discuss separate complaint file
4,Credit card or prepaid card,i had accounts with chase they closed my accounts upon review for reasons,account chase close account upon review reason


In [14]:
df_cleaned['Product'].unique()

array(['Credit card', '"Credit reporting', 'Bank account or service',
       'Debt collection', 'Credit card or prepaid card', 'Mortgage',
       'Credit reporting', 'Student loan', 'Checking or savings account',
       '"Money transfer', 'Money transfers', 'Payday loan',
       'Consumer Loan', 'Vehicle loan or lease', '"Payday loan',
       'Prepaid card', 'Other financial service', 'Virtual currency'],
      dtype=object)

In [15]:
df_cleaned['Product'] = df_cleaned['Product'].replace('"Credit reporting', 'Credit reporting')
df_cleaned['Product'] = df_cleaned['Product'].replace('Credit card', 'Credit card or prepaid card')
df_cleaned['Product'] = df_cleaned['Product'].replace('Prepaid card', 'Credit card or prepaid card')
df_cleaned['Product'] = df_cleaned['Product'].replace('Consumer Loan', 'Bank account or service')
df_cleaned['Product'] = df_cleaned['Product'].replace('Student loan', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('Payday loan', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('"Payday loan', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('Vehicle loan or lease', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('Mortgage', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('Debt collection', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('"Money transfer', 'Money transfers')
df_cleaned['Product'] = df_cleaned['Product'].replace('Checking or savings account', 'Bank account or service')
df_cleaned['Product'] = df_cleaned['Product'].replace('Virtual currency', 'Other financial service')
df_cleaned['Product'].unique()

array(['Credit card or prepaid card', 'Credit reporting',
       'Bank account or service', 'Loan and debt', 'Money transfers',
       'Other financial service'], dtype=object)

In [54]:

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score,recall_score,f1_score

In [55]:
label_encoder = LabelEncoder()
df_cleaned['product_labelled'] = label_encoder.fit_transform(df_cleaned['Product'])
label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print(label_mapping)
df_cleaned.head()

{0: 'Bank account or service', 1: 'Credit card or prepaid card', 2: 'Credit reporting', 3: 'Loan and debt', 4: 'Money transfers', 5: 'Other financial service'}


,Product,final_problem,lemmatised_description,product_labelled
0,Credit card or prepaid card,i am dissatisfied with the current outcome of a dispute that was initiated with discover card regarding a single transaction that occurred on in the amount of i have corresponded with discover card at least four times since which i have enclosed as an attachment to this complaint i believe that the credit card issuer has violated consumer protection laws by failing to implement the special rule for credit card purchase protection despite overwhelming paperwork evidence submitted by me that shows the merchant has conducted business in bad faith less favorable to the consumer i have sustained a monetary loss as a result of merchants bad faith and intent i have patiently utilized the internal discover card dispute process over the past three months with the credit card issuer always favoring the merchant i have repeatedly submitted irrefutable paperwork evidence that has shown that the merchant has conducted business in bad faith i have tried in good faith to address my complaint with the merchant and discover card but believe that i will not receive a favorable outcome,dissatisfied current outcome dispute initiate discover card regard single transaction occur amount correspond discover card least four time since enclose attachment complaint believe credit card issuer violate consumer protection law fail implement special rule credit card purchase protection despite overwhelming paperwork evidence submit show merchant conduct business bad faith less favorable consumer sustain monetary loss result merchant bad faith intent patiently utilize internal discover card dispute process past three month credit card issuer always favor merchant repeatedly submit irrefutable paperwork evidence show merchant conduct business bad faith try good faith address complaint merchant discover card believe receive favorable outcome,1
1,Credit reporting,incorrect information on your report,incorrect information report,2
2,Bank account or service,on my bank account balance with pnc bank was on the bank charged a overdraft fee with no other posted transactions,bank account balance pnc bank bank charge overdraft fee post transaction,0
3,Loan and debt,i received a collection notice and called right away i figured i had over looked it as we have many medical bills while on the phone i was told there were additional much older than the notice i called about i paid the one notice and said i needed to look at the date for the other one one of those were for my adult son and could not be discussed with me a separate complaint is being filed,receive collection notice call right away figure look many medical bill phone tell additional much old notice call pay one notice say need look date one one adult son could discuss separate complaint file,3
4,Credit card or prepaid card,i had accounts with chase they closed my accounts upon review for reasons,account chase close account upon review reason,1


In [56]:
X_train, X_val, y_train, y_val = train_test_split(df_cleaned["lemmatised_description"],
                                                  df_cleaned["product_labelled"],
                                                  test_size=0.2,
                                                  shuffle=True)
print(y_train)

20135     3
28130     2
80327     1
46667     2
18098     3
         ..
3756      3
64996     3
100444    1
97470     3
73929     0
Name: product_labelled, Length: 81979, dtype: int32


In [57]:

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) 
# print(X_train_vectors_tfidf)

In [58]:
import pickle
vectorizer_filename = 'tfidf_vectorizer.pkl'
with open(vectorizer_filename, 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [60]:
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train) 


LogisticRegression(C=10, solver='liblinear')

In [61]:
 
y_predict = lr_tfidf.predict(X_val_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]
print(classification_report(y_val, y_predict))
# r2_score(y_val, model.predict(X_val_vectors_tfidf)).round(2)


              precision    recall  f1-score   support

           0       0.88      0.79      0.83      2062
           1       0.82      0.78      0.80      1811
           2       0.96      0.94      0.95      5990
           3       0.92      0.96      0.94     10332
           4       0.90      0.90      0.90       289
           5       0.00      0.00      0.00        11

    accuracy                           0.92     20495
   macro avg       0.75      0.73      0.74     20495
weighted avg       0.92      0.92      0.92     20495



c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [62]:

model_filename = 'lr_tfidf_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(lr_tfidf, file)

print(f"Model saved to {model_filename}")

Model saved to lr_tfidf_model.pkl


In [63]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
dt = DecisionTreeClassifier().fit(X_train_vectors_tfidf, y_train)
predicted = dt.predict(X_val_vectors_tfidf)

print(classification_report(y_pred=predicted, y_true=y_val))

              precision    recall  f1-score   support

           0       0.79      0.76      0.77      2062
           1       0.74      0.71      0.72      1811
           2       0.94      0.93      0.94      5990
           3       0.91      0.93      0.92     10332
           4       0.88      0.87      0.87       289
           5       0.00      0.00      0.00        11

    accuracy                           0.89     20495
   macro avg       0.71      0.70      0.70     20495
weighted avg       0.89      0.89      0.89     20495



In [64]:
params = {
    'n_estimators': [10, 50, 100, 200, 500],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 4, 6, 8, 10],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 6, 8, 10],
    'max_features': [None, 'auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}
rfc = RandomForestClassifier(max_depth=10)
rfc.fit(X_train_vectors_tfidf , y_train)
predicted = rfc.predict(X_val_vectors_tfidf)

print(classification_report(y_pred=predicted, y_true=y_val))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2062
           1       0.00      0.00      0.00      1811
           2       1.00      0.79      0.88      5990
           3       0.66      1.00      0.79     10332
           4       0.00      0.00      0.00       289
           5       0.00      0.00      0.00        11

    accuracy                           0.73     20495
   macro avg       0.28      0.30      0.28     20495
weighted avg       0.62      0.73      0.66     20495



c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [65]:
# function to evaluate the model and display the results
def eval_model(y_test,y_pred,y_pred_proba,type='Training'):
    print(type,'results')
    print('Accuracy: ', accuracy_score(y_test,y_pred))
    print('Precision: ', precision_score(y_test,y_pred,average='weighted').round(2))
    print('Recall: ', recall_score(y_test,y_pred,average='weighted').round(2))
    print('F1 Score: ', f1_score(y_test,y_pred,average='weighted').round(2))
    if len(set(y_test)) == y_pred_proba.shape[1]:
        print('ROC AUC Score: ', roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class='ovr').round(2))
    else:
        print('ROC AUC Score: Not applicable')
    print('Classification Report: ', classification_report(y_test,y_pred))

    

# function to grid search the best parameters for the model
def run_model(model,param_grid):
    cv=StratifiedKFold(n_splits=2,shuffle=True,random_state=40)
    grid=GridSearchCV(model,param_grid={},cv=cv,scoring='f1_weighted',verbose=1,n_jobs=-1)
    grid.fit(X_train_vectors_tfidf,y_train)
    return grid.best_estimator_,grid.best_params_

In [66]:
params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300, 500,1000],
    'class_weight': [None, 'balanced']
}
model, best_params =run_model(LogisticRegression(),params)
print(best_params)
eval_model(y_train,model.predict(X_train_vectors_tfidf),model.predict_proba(X_train_vectors_tfidf),type='Training')
eval_model(y_val,model.predict(X_val_vectors_tfidf),model.predict_proba(X_val_vectors_tfidf),type='Test')
# unique_classes = set(model.predict(X_train_vectors_tfidf))
# print(unique_classes)
# number2= set(model.predict(X_val_vectors_tfidf))
# print(number2)
# r2_score(y_val, model.predict(X_val_vectors_tfidf)).round(2)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
{}
Training results
Accuracy:  0.9356420546725381
Precision:  0.94
Recall:  0.94
F1 Score:  0.93
ROC AUC Score:  0.99


c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

Classification Report:                precision    recall  f1-score   support

           0       0.92      0.82      0.87      8324
           1       0.88      0.82      0.85      7092
           2       0.97      0.95      0.96     23848
           3       0.93      0.97      0.95     41589
           4       0.92      0.91      0.91      1074
           5       0.00      0.00      0.00        52

    accuracy                           0.94     81979
   macro avg       0.77      0.75      0.76     81979
weighted avg       0.94      0.94      0.93     81979

Test results
Accuracy:  0.9200292754330325
Precision:  0.92
Recall:  0.92
F1 Score:  0.92
ROC AUC Score:  0.99
Classification Report:                precision    recall  f1-score   support

           0       0.89      0.79      0.84      2062
           1       0.84      0.78      0.81      1811
           2       0.97      0.93      0.95      5990
           3       0.91      0.97      0.94     10332
           4       0.91    

c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

{0: 'Bank account or service', 1: 'Credit card or prepaid card', 2: 'Credit reporting', 3: 'Loan and debt', 4: 'Money transfers', 5: 'Other financial service'}

In [67]:
input_data = ["Last time i checked my balance in my card it had 30000 rupees but today it got declined for a payment of 2000."]

X = tfidf_vectorizer.transform(input_data) 
model.predict(X)


array([1])